In [761]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
api_key = '02d2f30610d03110d218de7fed01b104a8a441a5'
import requests
import us
from bs4 import BeautifulSoup
import os

In [776]:
os.environ['CENSUS_API']

KeyError: 'CENSUS_API'

In [549]:
# Here is the plan:
# Develop a county by county ground game for the presidential election:
# Use pca to narrow a feature space of 16,000 census demographic fields and 2012 presidential election data to regress
# democratic and republican votes
# Train on 2013 us census data and 2012 election data by count features with target 
# democratic votes (as a % of votes) and same for republican
# Come up with a model that gives the best value for a democratic ground game in the nation
# take into account factors like cost differences, electoral college
# Make cool visualizations but no maps, everyone knows what the US looks like

# Get 2012 presidential data election data by county from here:
# https://github.com/huffpostdata/election-2012-results

# This gives me a county_id (FIPS #) which I plan to match up with census data somehow
# I know for a fact there are 3142 counties (supported below)
# The nan's are just because there are 15 spots for candidates, obama and romney are both in there for every county

# I really want to get better data here but still working on that

# probably will want to scrape 2008 from here:
#https://docs.google.com/spreadsheets/d/1gLzjUFBk9gtAPfZ-bNZVfFC1zNhGkY_WI_VD_OXHUYI/edit#gid=0
#    as another feature

In [772]:
# This gets the columns that I am going to need from the census later
link = 'http://api.census.gov/data/2014/acs5/profile/variables.html'
r = requests.get(link)
soup = BeautifulSoup(r.text)
table = soup.find('table')

name = '' 
label = ''
concept = ''
required = ''
predicate = ''
 
output = np.array(['1','2','3','4','5'])
for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 5:
        name = cells[0].find(text=True)
        label = cells[1].find(text=True)
        concept = cells[2].find(text=True)
        required = cells[3].find(text=True)
        predicate = cells[4].find(text=True)
        output = np.vstack((output,[name, label, concept, required, predicate]))
census_cols = pd.DataFrame(output)
census_cols.drop(0, inplace=True)
census_cols.columns = ['name', 'label', 'concept', 'required',' predicate']
household = []
for x in census_cols['label']:
    if 'HOUSEHOLD' in x:
        household.append(1)
    else: household.append(0)
census_cols['hosehold_bool'] = np.array(household) #1 if household stat
census_cols.head()

,name,label,concept,required,predicate,hosehold_bool
1,aihhtli,American Indian Trust Land/Hawaiian Home Land,Selectable Geographies,not required,int,0
2,aindn,American Indian Area/Alaska Native Area/Hawaii...,Selectable Geographies,not required,(not a predicate),0
3,aindnr,AINDNR FIPS code,Selectable Geographies,not required,(not a predicate),0
4,aitsce,AITSCE FIPS code,Selectable Geographies,not required,(not a predicate),0
5,anrc,Alaska Native Regional Corporation (FIPS),Selectable Geographies,not required,(not a predicate),0


In [777]:
census_cols['household'] = census_cols['label'].apply(lambda x: 1 if 'HOUSEHOLD' in x else 0)

In [824]:
names = census_cols.iloc[16:census_cols.shape[0] - 24]
col_names = np.unique(names['name'].apply(lambda x: x.strip('PEM')))
init_str = 'st&NAME'
col_str = init_str + ''.join('&' + x for x in col_names)
col_str

u'st&NAME&DP02PR_0001&DP02PR_0002&DP02PR_0003&DP02PR_0004&DP02PR_0005&DP02PR_0006&DP02PR_0007&DP02PR_0008&DP02PR_0009&DP02PR_0010&DP02PR_0011&DP02PR_0012&DP02PR_0013&DP02PR_0014&DP02PR_0015&DP02PR_0016&DP02PR_0017&DP02PR_0018&DP02PR_0019&DP02PR_0020&DP02PR_0021&DP02PR_0022&DP02PR_0023&DP02PR_0024&DP02PR_0025&DP02PR_0026&DP02PR_0027&DP02PR_0028&DP02PR_0029&DP02PR_0030&DP02PR_0031&DP02PR_0032&DP02PR_0033&DP02PR_0034&DP02PR_0035&DP02PR_0036&DP02PR_0037&DP02PR_0038&DP02PR_0039&DP02PR_0040&DP02PR_0041&DP02PR_0042&DP02PR_0043&DP02PR_0044&DP02PR_0045&DP02PR_0046&DP02PR_0047&DP02PR_0048&DP02PR_0049&DP02PR_0050&DP02PR_0051&DP02PR_0052&DP02PR_0053&DP02PR_0054&DP02PR_0055&DP02PR_0056&DP02PR_0057&DP02PR_0058&DP02PR_0059&DP02PR_0060&DP02PR_0061&DP02PR_0062&DP02PR_0063&DP02PR_0064&DP02PR_0065&DP02PR_0066&DP02PR_0067&DP02PR_0068&DP02PR_0069&DP02PR_0070&DP02PR_0071&DP02PR_0072&DP02PR_0073&DP02PR_0074&DP02PR_0075&DP02PR_0076&DP02PR_0077&DP02PR_0078&DP02PR_0079&DP02PR_0080&DP02PR_0081&DP02PR_0082&DP02PR

In [789]:
census_cols.head()

,name,label,concept,required,predicate,hosehold_bool,household
1,aihhtli,American Indian Trust Land/Hawaiian Home Land,Selectable Geographies,not required,int,0,0
2,aindn,American Indian Area/Alaska Native Area/Hawaii...,Selectable Geographies,not required,(not a predicate),0,0
3,aindnr,AINDNR FIPS code,Selectable Geographies,not required,(not a predicate),0,0
4,aitsce,AITSCE FIPS code,Selectable Geographies,not required,(not a predicate),0,0
5,anrc,Alaska Native Regional Corporation (FIPS),Selectable Geographies,not required,(not a predicate),0,0


In [773]:
# these make api calls to the census site to get data,
# Question if you read this and feel like pinging me, how do I get all the columns in the get_census_data call?
# I gathered the column names in the data frame above
# I probably have to do one column at a time to build up the dataframe
def get_county_data():
    search = 'http://api.census.gov/data/2014/acs5/profile?get=NAME&for=county:*'
    search += '&key=' + api_key
    df = pd.read_json(search, orient='DataFrame')
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    return df

def get_census_data(search_string):
    '''Column Choices: http://api.census.gov/data/2012/acs5/profile/variables.html'''
    '''E is total #, PE is percent, M is margin of error, PM percent margin'''
    search = 'http://api.census.gov/data/2014/acs5/profile?get=%s&for=county:*' % (search_string)
    search += '&key=' + api_key
    df = pd.read_json(search, orient='DataFrame')
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    return df

def get_pop_data(search_string):
    search ='http://api.census.gov/data/2010/sf1?get=%s&for=county:*' % search_string
    search += '&key=' + api_key
    df = pd.read_json(search, orient='DataFrame')
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    return df

In [768]:
pop = get_pop_data('NAME&P0010001')
df.to_csv()

In [769]:
pd.concat()

,NAME,P0010001,state,county
1,Autauga County,54571,01,001
2,Baldwin County,182265,01,003
3,Barbour County,27457,01,005
4,Bibb County,22915,01,007
5,Blount County,57322,01,009


In [828]:
census_data = get_census_data('st&NAME')
state = []

for x in census_data['NAME']:
    state.append(x.split(',')[1].strip())
census_data['STATE'] = np.array(state)
census_data = census_data[census_data['state'] != 'Puerto Rico']
census_data['INCITS'] = (census_data['st'] + census_data['county']).astype(str)
print df.shape
census_data.head(52)

(3142, 8)


,st,NAME,state,county,STATE,INCITS
1,01,"Autauga County, Alabama",01,001,Alabama,01001
2,01,"Baldwin County, Alabama",01,003,Alabama,01003
3,01,"Barbour County, Alabama",01,005,Alabama,01005
4,01,"Bibb County, Alabama",01,007,Alabama,01007
5,01,"Blount County, Alabama",01,009,Alabama,01009
6,01,"Bullock County, Alabama",01,011,Alabama,01011
7,01,"Butler County, Alabama",01,013,Alabama,01013
8,01,"Calhoun County, Alabama",01,015,Alabama,01015
9,01,"Chambers County, Alabama",01,017,Alabama,01017
10,01,"Cherokee County, Alabama",01,019,Alabama,01019


In [ ]:
census_cols.to_json()

In [621]:
# Get data on the 3142 counties because that's gotta be useful

wiki = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents'
r = requests.get(wiki)
soup = BeautifulSoup(r.text, 'lxml')
table = soup.find('table', { "class" : "wikitable sortable" })

INCITS = '' 
County = ''
State = ''
Pop_Rank = ''
Pop_2013 = ''
Core_Based_Statistical_Area = ''
Combined_Statistical_Area = ''
 
output = np.array(['1','2','3','4','5','6','7'])
for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 7:
        INCITS = cells[0].find(text=True)
        County = cells[1].find(text=True)
        State = cells[2].find(text=True)
        Pop_Rank = cells[3].find(text=True)
        Pop_2013 = cells[4].find(text=True)
        Core_Based_Statistical_Area = cells[5].find(text=True)
        Combined_Statistical_Area = cells[6].find(text=True)
        output = np.vstack((output,[INCITS, County, State, Pop_Rank, Pop_2013, Core_Based_Statistical_Area, Combined_Statistical_Area]))
        
counties = pd.DataFrame(output)
counties.drop(0, inplace=True)
counties.columns = ['INCITS', 'county', 'state', 'pop_rank',' pop_2013', 'core_stat_area', 'comb_stat_area']
print counties.shape
counties.head(10)

(3142, 7)


,INCITS,county,state,pop_rank,pop_2013,core_stat_area,comb_stat_area
1,01001,Autauga County,Alabama,911,"55,246","Montgomery, AL Metropolitan Statistical Area",None
2,01003,Baldwin County,Alabama,329,"195,540","Daphne-Fairhope-Foley, AL Micropolitan Statist...","Mobile-Daphne-Fairhope, AL Combined Statistica..."
3,01005,Barbour County,Alabama,1531,"27,076",None,None
4,01007,Bibb County,Alabama,1708,"22,512","Birmingham-Hoover, AL Metropolitan Statistical...","Birmingham-Hoover-Talladega, AL Combined Stati..."
5,01009,Blount County,Alabama,884,"57,872","Birmingham-Hoover, AL Metropolitan Statistical...","Birmingham-Hoover-Talladega, AL Combined Stati..."
6,01011,Bullock County,Alabama,2386,"10,639",None,None
7,01013,Butler County,Alabama,1828,"20,265",None,None
8,01015,Calhoun County,Alabama,522,"116,736","Anniston-Oxford-Jacksonville, AL Metropolitan ...",None
9,01017,Chambers County,Alabama,1323,"34,162","Valley, AL Micropolitan Statistical Area","Columbus-Auburn-Opelika, GA-AL Combined Statis..."
10,01019,Cherokee County,Alabama,1555,"26,203",None,None


In [364]:
# I'll probably want this later
voter_turnout = 'https://docs.google.com/spreadsheets/d/1EYjW8l4y-5xPbkTFjdjdpnxOCgVvB8rM_oqjtJhtQKY/pubhtml'

In [728]:
# Get 2012 election results and make a dataframce
def get_2012_election_results():
    files = os.listdir('data/election-2012-results/data')
    files.remove('ma_towns.csv')
    files.remove('ri_towns.csv')
    len(files)

    init = pd.DataFrame([[0, 'Alaska', 'Obama', 122640]]) # Data from wikipedia
    init = init.append([[0, 'Alaska', 'Romney', 164767]])
    init.columns = ['fips', 'county', 'candidate', 'votes']
    for filename in files[1:]:
        filename = 'data/election-2012-results/data/' + filename
        init = init.append(pd.read_csv(filename))

    obama = []
    for x in init.candidate:
        if 'obama' in x.lower():
            obama.append(1)
        else:
            obama.append(0)

    romney = []
    for x in init.candidate:
        if 'romney' in x.lower():
            romney.append(1)
        else:
            romney.append(0)
    init['Obama'] = obama
    init['Romney'] = romney
    init['other'] = ((1 + init['Obama'] + init['Romney']) - 2) * -1
    incits = []
    for x in init.fips:
        temp = str(x).split('.')[0]
        if len(temp) == 4:
            temp = '0' + temp
        incits.append(temp)
    
    init['INCITS'] = np.array(incits)
    return init
    
election = get_2012_election_results()

In [729]:
election.head(10000)

,fips,county,candidate,votes,Obama,Romney,other,INCITS
0,0,Alaska,Obama,122640,1,0,0,0
0,0,Alaska,Romney,164767,0,1,0,0
0,1001,Autauga,BARACK OBAMA / JOE BIDEN (D),6363,1,0,0,01001
1,1001,Autauga,MITT ROMNEY /PAUL RYAN (R),17379,0,1,0,01001
2,1001,Autauga,"VIRGIL H. GOODE, JR. / JAMES CLYMER (I)",31,0,0,1,01001
3,1001,Autauga,GARY JOHNSON / JIM GRAY (I),137,0,0,1,01001
4,1001,Autauga,JILL STEIN / CHERI HONKALA (I),22,0,0,1,01001
5,1001,Autauga,WRITE-IN,41,0,0,1,01001
6,1003,Baldwin,BARACK OBAMA / JOE BIDEN (D),18424,1,0,0,01003
7,1003,Baldwin,MITT ROMNEY /PAUL RYAN (R),66016,0,1,0,01003


In [731]:
def join_election_census(election, census_data):
    obama = []
    romney = []
    for incit in census_data.INCITS:
        temp1 = election[(election['INCITS'] == incit) & election['Obama'] == 1]['votes'].values
        temp2 = election[(election['INCITS'] == incit) & election['Romney'] == 1]['votes'].values
        if len(temp1) == 0:
            temp1 = [0]
            temp2 = [0]
        obama.append(temp1[0])
        romney.append(temp2[0])

    obama = np.array(obama).flatten()
    romney = np.array(romney).flatten()
    census_data['Obama_votes'] = obama
    census_data['Romney_votes'] = romney
    return census_data

,st,NAME,DP02_0001E,DP02_0001M,DP02_0001PE,state,county,INCITS,Obama_votes,Romney_votes
1,01,"Autauga County, Alabama",20304,458,20304,Alabama,001,01001,6363,17379
2,01,"Baldwin County, Alabama",73058,1241,73058,Alabama,003,01003,18424,66016
3,01,"Barbour County, Alabama",9145,311,9145,Alabama,005,01005,5912,5550
4,01,"Bibb County, Alabama",7078,390,7078,Alabama,007,01007,2202,6132
5,01,"Blount County, Alabama",20934,399,20934,Alabama,009,01009,2970,20757


In [570]:
census_data[census_data['state'] == 'Alaska']
alaska = census_data[census_data['state'] == 'Alaska'].groupby('state').sum()

In [571]:
alaska

,fips,Obama_votes,Romney_votes
state,,,
Alaska,62452,0,0


In [604]:
def load_offices(fname):
    df = pd.read_csv(fname, "|")
    output = []
    for x in xrange(df.shape[0]):
        output.append(df['State'].iloc[x] + '_' + df['County'].iloc[x].strip('"'))
    df['county_id'] = np.array(output)
    output = df[['county_id','Name']].groupby('county_id').count()
    output.columns = ['Count']
    return output

In [607]:
obama_offices = load_offices('data/Obama_Office_Locations_Parsed_Cleaned.csv')
romney_offices = load_offices('data/Romney_Office_Locations_Parsed_Cleaned.csv')

In [609]:
romney_offices.head()

,Count
county_id,
CO_Adams,1
CO_Arapahoe,2
CO_Boulder,1
CO_Denver,1
CO_Douglas,1


In [624]:
df_joined = pd.merge(census_data, counties, on='INCITS')

In [654]:
match = np.unique(census_data.INCITS) == np.unique(counties.INCITS)
idx = np.argsort(match)
print len(match) - sum(match)

6


In [666]:
print counties.INCITS[idx][0:7]
print census_data.INCITS[idx][0:7]

2412    46101
2413    46102
2414    46103
2415    46105
2416    46107
2417    46109
0         NaN
Name: INCITS, dtype: object
2412    46101
2413    46103
2414    46105
2415    46107
2416    46109
2417    46111
0         NaN
Name: INCITS, dtype: object


In [706]:
census_data.head(500)

,st,NAME,DP02_0001E,DP02_0001M,DP02_0001PE,state,county,INCITS
1,01,"Autauga County, Alabama",20304,458,20304,Alabama,001,01001
2,01,"Baldwin County, Alabama",73058,1241,73058,Alabama,003,01003
3,01,"Barbour County, Alabama",9145,311,9145,Alabama,005,01005
4,01,"Bibb County, Alabama",7078,390,7078,Alabama,007,01007
5,01,"Blount County, Alabama",20934,399,20934,Alabama,009,01009
6,01,"Bullock County, Alabama",3746,216,3746,Alabama,011,01011
7,01,"Butler County, Alabama",8253,252,8253,Alabama,013,01013
8,01,"Calhoun County, Alabama",45348,705,45348,Alabama,015,01015
9,01,"Chambers County, Alabama",13901,393,13901,Alabama,017,01017
10,01,"Cherokee County, Alabama",11726,451,11726,Alabama,019,01019


In [692]:
election.head()

,fips,county,candidate,votes,Obama,Romney,other,INCITS
0,0.0,Alaska,Obama,122640,1,0,0,0.0
0,0.0,Alaska,Romney,164767,0,1,0,0.0
0,1001.0,Autauga,BARACK OBAMA / JOE BIDEN (D),6363,1,0,0,1001.0
1,1001.0,Autauga,MITT ROMNEY /PAUL RYAN (R),17379,0,1,0,1001.0
2,1001.0,Autauga,"VIRGIL H. GOODE, JR. / JAMES CLYMER (I)",31,0,0,1,1001.0


In [685]:
a = '1'

In [687]:
a = '0' + a

'01'